In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Khám phá dataset Google Play Store Apps
#### lấy mẫu 500k từ dataset hơn 2 triệu apps
https://www.kaggle.com/datasets/gauthamp10/google-playstore-apps

In [85]:
df = pd.read_csv('Google-Playstore-Apps-300k.csv')
print(df.shape)
print(df.columns)
df.head(5)

(300000, 24)
Index(['App Name', 'App Id', 'Category', 'Rating', 'Rating Count', 'Installs',
       'Minimum Installs', 'Maximum Installs', 'Free', 'Price', 'Currency',
       'Size', 'Minimum Android', 'Developer Id', 'Developer Website',
       'Developer Email', 'Released', 'Last Updated', 'Content Rating',
       'Privacy Policy', 'Ad Supported', 'In App Purchases', 'Editors Choice',
       'Scraped Time'],
      dtype='object')


,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,be.MOBILISED,com.hastobe.bemobilised,Maps & Navigation,0.0,0.0,100+,100.0,274,True,0.0,...,http://www.has-to-be.com,support@has-to-be.com,"Feb 9, 2020","May 14, 2021",Everyone,https://has-to-be.com/de/ueber-uns/datenschutz...,False,False,False,2021-06-15 20:19:36
1,OTENTIK Discovery FR,com.camineo.otentikdiscoveryDLFR,Travel & Local,3.4,5.0,500+,500.0,949,True,0.0,...,http://www.hotels-attitude.com,info@hotels-attitude.com,"Nov 28, 2016","Oct 30, 2019",Everyone,http://camineo.com/w7WEW99wBsdTH2FuyOeTQACM6mc...,False,False,False,2021-06-15 20:19:36
2,Nepali Congress,np.com.NepaliCongress,Social,0.0,0.0,100+,100.0,336,True,0.0,...,https://nepalicongress.org,info@nepalicongress.org,"May 31, 2021","May 31, 2021",Everyone,https://nepalicongress.org/page/privacy-policy,False,False,False,2021-06-15 20:19:36
3,CUBIC RUNNER,jp.ga_software.cubicrunner,Arcade,0.0,0.0,"1,000+",1000.0,1068,True,0.0,...,http://ga-software.blogspot.jp/,gasoftwarejp@gmail.com,"Apr 1, 2015","Apr 16, 2015",Everyone,NaN,True,True,False,2021-06-15 20:19:37
4,Creepy Story,com.creepypasta.club,Entertainment,0.0,0.0,10+,10.0,22,True,0.0,...,NaN,andizulmai@gmail.com,"May 28, 2019","May 30, 2019",Teen,http://hidayah.club/creepystory_privacy_policy...,True,False,False,2021-06-15 20:19:37


### Chọn các cols cần thiết cho phân tích

In [86]:
data = df[["App Name", "Category", "Rating", "Rating Count","Minimum Installs","Maximum Installs", "Free","Price","Size","Released","Last Updated","Content Rating","Ad Supported"]]
data.head(3)


,App Name,Category,Rating,Rating Count,Minimum Installs,Maximum Installs,Free,Price,Size,Released,Last Updated,Content Rating,Ad Supported
0,be.MOBILISED,Maps & Navigation,0.0,0.0,100.0,274,True,0.0,15M,"Feb 9, 2020","May 14, 2021",Everyone,False
1,OTENTIK Discovery FR,Travel & Local,3.4,5.0,500.0,949,True,0.0,2.9M,"Nov 28, 2016","Oct 30, 2019",Everyone,False
2,Nepali Congress,Social,0.0,0.0,100.0,336,True,0.0,24M,"May 31, 2021","May 31, 2021",Everyone,False


### Check sơ lượt dataset


In [87]:
print(data.shape)
print(data.info())

(300000, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App Name          299999 non-null  object 
 1   Category          300000 non-null  object 
 2   Rating            297032 non-null  float64
 3   Rating Count      297032 non-null  float64
 4   Minimum Installs  299983 non-null  float64
 5   Maximum Installs  300000 non-null  int64  
 6   Free              300000 non-null  bool   
 7   Price             300000 non-null  float64
 8   Size              299974 non-null  object 
 9   Released          290654 non-null  object 
 10  Last Updated      300000 non-null  object 
 11  Content Rating    300000 non-null  object 
 12  Ad Supported      300000 non-null  bool   
dtypes: bool(2), float64(4), int64(1), object(6)
memory usage: 25.7+ MB
None


In [88]:
print(data.describe(include='all'))

           App Name   Category         Rating  Rating Count  Minimum Installs  \
count        299999     300000  297032.000000  2.970320e+05      2.999830e+05   
unique       294629         48            NaN           NaN               NaN   
top     Tic Tac Toe  Education            NaN           NaN               NaN   
freq             47      31129            NaN           NaN               NaN   
mean            NaN        NaN       2.199983  2.206330e+03      1.812697e+05   
std             NaN        NaN       2.106347  6.655720e+04      1.425936e+07   
min             NaN        NaN       0.000000  0.000000e+00      0.000000e+00   
25%             NaN        NaN       0.000000  0.000000e+00      5.000000e+01   
50%             NaN        NaN       2.900000  6.000000e+00      5.000000e+02   
75%             NaN        NaN       4.300000  4.200000e+01      5.000000e+03   
max             NaN        NaN       5.000000  1.429116e+07      5.000000e+09   

        Maximum Installs   

### cleaning data
#### cleaning null

In [89]:
# App Name: replace = 'Not name'
data['App Name'] = data['App Name'].replace(np.nan, "Not name")
# Rating & Rating Count: replace = 0
data[['Rating','Rating Count']] = data[['Rating','Rating Count']].replace(np.nan, 0)
# Minimum Installs: replace = Maximum Installs / 2
data['Minimum Installs'].fillna(data['Maximum Installs']/2, inplace=True)
# Size: replace = '0M'
data['Size'].fillna('0M', inplace=True)
# Released: replace = Last Updated
data['Released'].fillna(data['Last Updated'], inplace=True)

/tmp/ipykernel_6143/1041435560.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['App Name'] = data['App Name'].replace(np.nan, "Not name")
/tmp/ipykernel_6143/1041435560.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['Rating','Rating Count']] = data[['Rating','Rating Count']].replace(np.nan, 0)
/tmp/ipykernel_6143/1041435560.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App Name          300000 non-null  object 
 1   Category          300000 non-null  object 
 2   Rating            300000 non-null  float64
 3   Rating Count      300000 non-null  float64
 4   Minimum Installs  300000 non-null  float64
 5   Maximum Installs  300000 non-null  int64  
 6   Free              300000 non-null  bool   
 7   Price             300000 non-null  float64
 8   Size              300000 non-null  object 
 9   Released          300000 non-null  object 
 10  Last Updated      300000 non-null  object 
 11  Content Rating    300000 non-null  object 
 12  Ad Supported      300000 non-null  bool   
dtypes: bool(2), float64(4), int64(1), object(6)
memory usage: 25.7+ MB


#### duplicate

In [91]:
data.loc[data.duplicated()].value_counts()

Series([], dtype: int64)

#### wrong format
check wrong format

In [107]:

print(data['Size'].apply(lambda x: type(x)).unique())
# => [<class 'str'> <class 'float'>]
data['Unit byte'] = data['Size'].apply(lambda x: x if isinstance(x, (int, float))
                                                    else 0 if x == 'Varies with device' 
                                                    else float(x[:-1].replace(",", "")) if x[-1] == 'M'
                                                    else float(x[:-1].replace(",", ""))*1024 if x[-1] == "G"
                                                    else float(x[:-1].replace(",", "")) if x[-1] == "k"
                                                    else 0)
# data['Unit byte'] = data['Size'].str.slice(-1)
# data['Unit byte'].value_counts().index.to_list()

[<class 'str'>]


/tmp/ipykernel_6143/984833333.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Unit byte'] = data['Size'].apply(lambda x: x if isinstance(x, (int, float))


In [98]:
# df['Size'].apply(lambda x: type(x) if x else 'not')
data.loc[not isinstance(['Size'],(int, float))]
# type(df['Size'][0])

KeyError: 'True: boolean label can not be used without a boolean index'

In [102]:
# data.loc[data['Unit byte'] == 'e'].head(5)
data.head(5)

,App Name,Category,Rating,Rating Count,Minimum Installs,Maximum Installs,Free,Price,Size,Released,Last Updated,Content Rating,Ad Supported,Unit byte
0,be.MOBILISED,Maps & Navigation,0.0,0.0,100.0,274,True,0.0,15M,"Feb 9, 2020","May 14, 2021",Everyone,False,15.0
1,OTENTIK Discovery FR,Travel & Local,3.4,5.0,500.0,949,True,0.0,2.9M,"Nov 28, 2016","Oct 30, 2019",Everyone,False,2.9
2,Nepali Congress,Social,0.0,0.0,100.0,336,True,0.0,24M,"May 31, 2021","May 31, 2021",Everyone,False,24.0
3,CUBIC RUNNER,Arcade,0.0,0.0,1000.0,1068,True,0.0,16M,"Apr 1, 2015","Apr 16, 2015",Everyone,True,16.0
4,Creepy Story,Entertainment,0.0,0.0,10.0,22,True,0.0,6.1M,"May 28, 2019","May 30, 2019",Teen,True,6.1
